[Open In Colab](https://colab.research.google.com/github/joelbenjohn/pyCUFSM/blob/master/Validation.ipynb)

In [ ]:
!rm -rf /content/pyCUFSM
!git clone https://github.com/joelbenjohn/pyCUFSM

import sys
sys.path.insert(1, '/content/pyCUFSM')

In [ ]:
from google.colab import files
import os
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
#          os.rename(k, 'a.mat')
files = getLocalFiles()

In [ ]:
import numpy as np
from scipy import io
import pycufsm.fsm as fsm
import pycufsm.cfsm as cfsm
import pycufsm.helpers as helpers
import pycufsm.plotters as plotters
import pycufsm.preprocess as preprocess
from cutwp_prop2 import cutwp_prop2
for k, v in files.items():
    file = io.loadmat(k)
    cufsm_input = helpers.load_mat(file)
    sect_props = cutwp_prop2(cufsm_input['nodes'][:, 1:3], cufsm_input['elements'][:, 1:4])
    gbt_con = {
        'glob': GBTcon['glob'],
        'dist': GBTcon['dist'],
        'local': GBTcon['local'],
        'other': GBTcon['other'],
        'o_space': 1,
        'couple': 1,
        'orth': 2,
        'norm': 0,
    }
    signature, curve, basis = fsm.strip(props=cufsm_input['props'],
                                 nodes=cufsm_input['nodes'],
                                 elements=cufsm_input['elements'],
                                 lengths=cufsm_input['lengths'],
                                 springs=cufsm_input['springs'],
                                 constraints=cufsm_input['constraints'],
                                 gbt_con=gbt_con,
                                 b_c='S-S',
                                 m_all=np.ones((len(cufsm_input['lengths']), 1)),
                                 n_eigs=10,
                                 sect_props=sect_props)
    plt.semilogx(cufsm_input['lengths'], np.array(signature), 'k', label = 'python')
    plt.semilogx(cufsm_input['lengths'], np.array(curve)[:, 0], 'ko', label = 'matlab')
    plt.xlim((1, np.max(cufsm_input['lengths'])))
    ubound = np.min([np.max(signature), 3*np.median(signature)])
    plt.ylim((0, ubound))
    plt.xlabel('half-wavelength (in)')
    plt.ylabel('Pcr/Py')
    plt.show()
    
    
    
    